# VLM Benchmark for Object Property Abstraction

This notebook implements a benchmark for evaluating Vision Language Models (VLMs) on object property abstraction and visual question answering (VQA) tasks. The benchmark includes three types of questions:

1. Direct Recognition
2. Property Inference
3. Counterfactual Reasoning

And three types of images:
- REAL
- ANIMATED
- AI GENERATED

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [1]:
# Install required packages
# %pip install transformers torch Pillow tqdm bitsandbytes accelerate

In [2]:
%pip install qwen-vl-utils flash-attn #--no-build-isolation

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import required libraries
import torch
import json
from pathlib import Path
from PIL import Image
import gc
import re
from tqdm import tqdm
from typing import List, Dict, Any
from qwen_vl_utils import process_vision_info
import time

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

/var/scratch/ave303/anaconda3/envs/op_bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


## Benchmark Tester Class

This class handles the evaluation of models against our benchmark.

In [4]:
# class BenchmarkTester:
#     def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         with open(benchmark_path, 'r') as f:
#             self.benchmark = json.load(f)
#         self.data_dir = data_dir
    
#     def format_question(self, question, model_name):
#         """Format a question for the model."""

#         if model_name=="blip2":
#             return f"Question: {question['question']} Answer:"
#         else:
#             return f"Question: {question['question']} Answer with a number and list of objects. Answer:"

#     def clean_answer(self, answer):
#         """Clean the model output to extract just the number."""
#         # Remove any text that's not a number
#         # import re
#         # numbers = re.findall(r'\d+', answer)
#         # if numbers:
#         #     return numbers[0]  # Return the first number found
#         # return answer
#         """Extract number and reasoning from the model's answer."""
#         # Try to extract number and reasoning using regex
#         import re
#         pattern = r'(\d+)\s*\[(.*?)\]'
#         match = re.search(pattern, answer)
        
#         if match:
#             number = match.group(1)
#             objects = [obj.strip() for obj in match.group(2).split(',')]
#             return {
#                 "count": number,
#                 "reasoning": objects
#             }
#         else:
#             # Fallback if format isn't matched
#             numbers = re.findall(r'\d+', answer)
#             return {
#                 "count": numbers[0] if numbers else "0",
#                 "reasoning": []
#             }

#     def model_generation(self, model_name, model, inputs, processor):
#         """Generate answer and decode."""
#         outputs = None  # Initialize outputs to None
        
#         if model_name=="smolVLM2":
#             outputs = model.generate(**inputs, do_sample=False, max_new_tokens=64)
#             answer = processor.batch_decode(
#                 outputs,
#                 skip_special_tokens=True,
#             )[0]
#         elif model_name=="Qwen2.5-VL":
#             outputs = model.generate(**inputs, max_new_tokens=50)
#             outputs = [
#                 out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, outputs)
#             ]
#             answer = processor.batch_decode(
#                 outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False
#             )[0]
#         else:
#             print(f"Warning: Unknown model name '{model_name}' in model_generation.")
#             answer = ""  # Return an empty string

#         return answer, outputs
    
#     def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
#         results = []
#         print(f"\nEvaluating {model_name}...")
#         print(f"Using device: {self.device}")
        
#         # Force garbage collection before starting
#         gc.collect()
#         torch.cuda.empty_cache()

#         try:
#             images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
#             total_images = len(images)
            
#             for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
#                 try:
#                     print(f"\nProcessing image {idx+1}/{total_images}: {image_data['image_id']}")
#                     image_path = Path(self.data_dir)/image_data['path']
#                     if not image_path.exists():
#                         print(f"Warning: Image not found at {image_path}")
#                         continue
                    
#                     # Load and preprocess image
#                     image = Image.open(image_path).convert("RGB")
#                     image_results = []  # Store results for current image
                    
#                     for question in image_data['questions']:
#                         try:
#                             # prompt = self.format_question(question, model_name)
#                             print(f"Question: {question['question']}")

#                             messages = [
#                                 {
#                                     "role": "user",
#                                     "content": [
#                                         {"type": "image", "image": image},
#                                         # {"type": "text", "text": f"{question['question']} Answer format: total number(numerical) objects(within square brackets)"},
#                                         # {"type": "text", "text": f"{question['question']} Provide just the total count and the list of objects in the given format \n Format: number [objects]"},
#                                         # {"type": "text", "text": f"{question['question']} Answer Format: number [objects]"},
#                                         {"type": "text", "text": f"{question["question"]} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]"}
#                                     ]
#                                 },
#                             ]
                            
#                             # Clear cache before processing each question
#                             torch.cuda.empty_cache()
                            
#                             # Process image and text
#                             # inputs = processor(images=image, text=prompt, return_tensors="pt").to(self.device)
#                             if model_name=="smolVLM2":
#                                 inputs = processor.apply_chat_template(
#                                     messages,
#                                     add_generation_prompt=True,
#                                     tokenize=True,
#                                     return_dict=True,
#                                     return_tensors="pt",
#                                 ).to(model.device, dtype=torch.float16)
#                             else:
                                
#                                 text = processor.apply_chat_template(
#                                     messages, tokenize=False, add_generation_prompt=True
#                                 )
#                                 # image_inputs, video_inputs = process_vision_info(messages)
#                                 inputs = processor(
#                                     text=text,
#                                     images=image,
#                                     videos=None,
#                                     padding=True,
#                                     return_tensors="pt",
#                                 ).to("cuda")
                            
#                             # Generate answer with better settings
#                             with torch.no_grad():
#                                 answer, outputs = self.model_generation(model_name, model, inputs, processor)    #call for model.generate
        
#                             cleaned_answer = self.clean_answer(answer)
                            
#                             image_results.append({
#                                 "image_id": image_data["image_id"],
#                                 "image_type": image_data["image_type"],
#                                 "question_id": question["id"],
#                                 "question": question["question"],
#                                 "ground_truth": question["answer"],
#                                 "model_answer": cleaned_answer["count"],
#                                 "model_reasoning": cleaned_answer["reasoning"],
#                                 "raw_answer": answer,  # Keep raw answer for debugging
#                                 "property_category": question["property_category"]
#                             })
                            
#                             # Clear memory
#                             del outputs, inputs
#                             torch.cuda.empty_cache()
                            
#                         except Exception as e:
#                             print(f"Error processing question: {str(e)}")
#                             continue
                    
#                     # Add results from this image
#                     results.extend(image_results)
                    
#                     # Save intermediate results only every 2 images or if it's the last image
#                     if (idx + 1) % 2 == 0 or idx == total_images - 1:
#                         with open(f"{save_path}_checkpoint.json", 'w') as f:
#                             json.dump(results, f, indent=4)
                            
#                 except Exception as e:
#                     print(f"Error processing image {image_data['image_id']}: {str(e)}")
#                     continue
            
#             # Save final results
#             if results:
#                 with open(save_path, 'w') as f:
#                     json.dump(results, f, indent=4)
            
#         except Exception as e:
#             print(f"An error occurred during evaluation: {str(e)}")
#             if results:
#                 with open(f"{save_path}_error_state.json", 'w') as f:
#                     json.dump(results, f, indent=4)
        
#         return results

In [5]:
# import torch
# import json
# from pathlib import Path
# from PIL import Image
# import gc
# import re
# import time
# from tqdm import tqdm
# from typing import List, Dict, Any
# import psutil
# import os

# class BenchmarkTester:
#     def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         with open(benchmark_path, 'r') as f:
#             self.benchmark = json.load(f)
#         self.data_dir = data_dir
        
#         # Set memory optimization environment variables
#         os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
        
#         # Memory monitoring
#         self.max_memory_allocated = 0
#         self.memory_threshold = 0.70  # 70% of GPU memory as threshold

#     def get_gpu_memory_info(self):
#         """Get current GPU memory usage information."""
#         if torch.cuda.is_available():
#             allocated = torch.cuda.memory_allocated() / 1024**3  # GB
#             reserved = torch.cuda.memory_reserved() / 1024**3    # GB
#             max_memory = torch.cuda.max_memory_allocated() / 1024**3  # GB
#             total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
            
#             return {
#                 'allocated': allocated,
#                 'reserved': reserved,
#                 'max_allocated': max_memory,
#                 'total': total_memory,
#                 'free': total_memory - allocated,
#                 'usage_percent': (allocated / total_memory) * 100
#             }
#         return None

#     def aggressive_memory_cleanup(self):
#         """Perform aggressive memory cleanup - alias for ultra_aggressive_memory_cleanup."""
#         self.ultra_aggressive_memory_cleanup()

#     def ultra_aggressive_memory_cleanup(self):
#         """Perform ultra-aggressive memory cleanup including model cache clearing."""
#         # Clear Python garbage collector multiple times
#         for _ in range(5):
#             gc.collect()
        
#         if torch.cuda.is_available():
#             # Force synchronize all streams
#             torch.cuda.synchronize()
#             # Clear all cached memory
#             torch.cuda.empty_cache()
#             # Reset peak memory stats
#             torch.cuda.reset_peak_memory_stats()
#             # Force memory defragmentation
#             torch.cuda.memory.empty_cache()
#             # Another sync to ensure completion
#             torch.cuda.synchronize()
            
#         # Force system memory cleanup
#         import ctypes
#         libc = ctypes.CDLL("libc.so.6")
#         libc.malloc_trim(0)

#     def check_available_memory_and_restart_if_needed(self):
#         """Check if we need to recommend model restart due to fragmentation."""
#         memory_info = self.get_gpu_memory_info()
#         if memory_info:
#             # If allocated is much less than reserved, we have fragmentation
#             fragmentation_ratio = memory_info['reserved'] / max(memory_info['allocated'], 0.1)
#             if fragmentation_ratio > 2.0 and memory_info['usage_percent'] > 80:
#                 print(f"⚠️  Severe memory fragmentation detected (fragmentation ratio: {fragmentation_ratio:.2f})")
#                 print("Consider restarting the Python process to defragment GPU memory")
#                 return False
#         return True

#     def resize_image_if_needed(self, image, max_size=(512, 512)):
#         """Resize image aggressively to prevent memory issues."""
#         original_size = image.size
        
#         # Always resize to max_size to ensure consistent memory usage
#         # Calculate aspect ratio preserving resize
#         ratio = min(max_size[0] / original_size[0], max_size[1] / original_size[1])
#         new_size = (int(original_size[0] * ratio), int(original_size[1] * ratio))
        
#         print(f"Resizing image from {original_size} to {new_size}")
#         # Use NEAREST for fastest processing and lowest memory
#         image = image.resize(new_size, Image.Resampling.NEAREST)
        
#         return image

#     def check_memory_before_processing(self, image_id, skip_if_high=True):
#         """Check if we have enough memory before processing with option to skip."""
#         memory_info = self.get_gpu_memory_info()
#         if memory_info and memory_info['usage_percent'] > self.memory_threshold * 100:
#             print(f"Warning: High memory usage ({memory_info['usage_percent']:.1f}%) before processing {image_id}")
#             self.ultra_aggressive_memory_cleanup()
            
#             # Check again after cleanup
#             memory_info = self.get_gpu_memory_info()
#             if memory_info['usage_percent'] > self.memory_threshold * 100:
#                 print(f"Critical: Still high memory usage ({memory_info['usage_percent']:.1f}%) after cleanup")
                
#                 # Check for fragmentation issues
#                 if not self.check_available_memory_and_restart_if_needed():
#                     return False
                    
#                 if skip_if_high:
#                     print(f"Skipping {image_id} due to insufficient memory")
#                     return False
#         return True

#     def clean_answer(self, answer):
#         """Extract number and reasoning from the model's answer."""
#         import re
#         pattern = r'(\d+)\s*\[(.*?)\]'
#         match = re.search(pattern, answer)
        
#         if match:
#             number = match.group(1)
#             objects = [obj.strip() for obj in match.group(2).split(',')]
#             return {
#                 "count": number,
#                 "reasoning": objects
#             }
#         else:
#             numbers = re.findall(r'\d+', answer)
#             return {
#                 "count": numbers[0] if numbers else "0",
#                 "reasoning": []
#             }

#     def model_generation(self, model_name, model, inputs, processor):
#         """Generate answer with memory-optimized inference."""
#         outputs = None
        
#         try:
#             if model_name == "Qwen2.5-VL":
#                 # Use gradient checkpointing and mixed precision if available
#                 with torch.cuda.amp.autocast(enabled=True):
#                     outputs = model.generate(
#                         **inputs, 
#                         max_new_tokens=200,
#                         do_sample=False,
#                         temperature=None,
#                         top_p=None,
#                         top_k=None,
#                         num_beams=1,
#                         early_stopping=False,
#                         pad_token_id=processor.tokenizer.pad_token_id,
#                         eos_token_id=processor.tokenizer.eos_token_id,
#                         use_cache=False,  # Disable KV cache to save memory
#                     )
                
#                 outputs = [
#                     out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, outputs)
#                 ]
#                 answer = processor.batch_decode(
#                     outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False
#                 )[0]
#             else:
#                 print(f"Warning: Unknown model name '{model_name}' in model_generation.")
#                 answer = ""

#             return answer, outputs
            
#         except torch.cuda.OutOfMemoryError as e:
#             print(f"CUDA OOM during generation: {e}")
#             # Aggressive cleanup and retry once
#             self.aggressive_memory_cleanup()
#             raise e

#     def process_single_question(self, model_name, model, processor, image, question, image_id):
#         """Process a single question with extreme memory optimization."""
#         try:
#             # Ultra-aggressive pre-check
#             if not self.check_memory_before_processing(f"{image_id}_q{question['id']}", skip_if_high=False):
#                 raise RuntimeError("Insufficient GPU memory after cleanup")

#             # Create a minimal image copy to avoid references
#             image_copy = image.copy()
            
#             messages = [
#                 {
#                     "role": "user",
#                     "content": [
#                         {"type": "image", "image": image_copy},
#                         {"type": "text", "text": f"{question['question']} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]"}
#                     ]
#                 },
#             ]
            
#             # Process with maximum memory optimization
#             text = processor.apply_chat_template(
#                 messages, tokenize=False, add_generation_prompt=True
#             )
            
#             # Monitor memory before tokenization
#             memory_before = self.get_gpu_memory_info()
#             if memory_before and memory_before['usage_percent'] > 75:
#                 print(f"⚠️  Memory usage high before tokenization: {memory_before['usage_percent']:.1f}%")
#                 self.ultra_aggressive_memory_cleanup()
            
#             # Process inputs with minimal memory footprint
#             inputs = processor(
#                 text=text,
#                 images=image_copy,
#                 videos=None,
#                 padding=True,
#                 return_tensors="pt",
#             )
            
#             # Move to device only when needed
#             inputs = {k: v.to(self.device) if hasattr(v, 'to') else v for k, v in inputs.items()}
            
#             # Delete image copy immediately
#             del image_copy, messages
#             self.ultra_aggressive_memory_cleanup()
            
#             # Monitor memory before generation
#             memory_before_gen = self.get_gpu_memory_info()
#             if memory_before_gen:
#                 print(f"Memory before generation: {memory_before_gen['usage_percent']:.1f}%")
#                 if memory_before_gen['usage_percent'] > 85:
#                     raise RuntimeError(f"Memory too high for generation: {memory_before_gen['usage_percent']:.1f}%")
            
#             # Generate with maximum memory efficiency
#             with torch.no_grad():
#                 with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16):
#                     answer, outputs = self.model_generation(model_name, model, inputs, processor)
            
#             cleaned_answer = self.clean_answer(answer)
            
#             # Immediate and thorough cleanup
#             del outputs, inputs
#             self.ultra_aggressive_memory_cleanup()
            
#             return {
#                 "question_id": question["id"],
#                 "question": question["question"],
#                 "ground_truth": question["answer"],
#                 "model_answer": cleaned_answer["count"],
#                 "model_reasoning": cleaned_answer["reasoning"],
#                 "raw_answer": answer,
#                 "property_category": question["property_category"]
#             }
            
#         except (torch.cuda.OutOfMemoryError, RuntimeError) as e:
#             print(f"Error processing question {question['id']}: {e}")
#             self.ultra_aggressive_memory_cleanup()
#             raise e

#     def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
#         results = []
        
#         # Initialize tracking variables
#         successful_images = []
#         failed_images = []
#         total_questions_processed = 0
#         total_questions_failed = 0
        
#         print(f"\nEvaluating {model_name}...")
#         print(f"Using device: {self.device}")
        
#         # Initial memory cleanup
#         self.ultra_aggressive_memory_cleanup()
        
#         # Print initial memory status
#         memory_info = self.get_gpu_memory_info()
#         if memory_info:
#             print(f"Initial GPU memory: {memory_info['usage_percent']:.1f}% used")

#         try:
#             images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
#             total_images = len(images)
            
#             for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
#                 image_start_time = time.time()
#                 current_image_questions_failed = 0
#                 current_image_questions_total = 0
                
#                 try:
#                     image_path = Path(self.data_dir) / image_data['path']
#                     if not image_path.exists():
#                         failed_images.append({
#                             'image_id': image_data['image_id'],
#                             'image_type': image_data.get('image_type', 'unknown'),
#                             'error_type': 'file_not_found',
#                             'error_message': f'Image not found at {image_path}'
#                         })
#                         continue
                    
#                     # Load and preprocess image with size control
#                     image = Image.open(image_path).convert("RGB")
#                     print(f"Original image size: {image.size}")
                    
#                     # Resize aggressively - much smaller images
#                     image = self.resize_image_if_needed(image, max_size=(384, 384))
                    
#                     image_results = []
                    
#                     # Process questions one by one with memory monitoring
#                     for question_idx, question in enumerate(image_data['questions']):
#                         current_image_questions_total += 1
#                         total_questions_processed += 1
                        
#                         try:
#                             # Process single question
#                             question_result = self.process_single_question(
#                                 model_name, model, processor, image, question, image_data['image_id']
#                             )
                            
#                             # Add image metadata
#                             question_result.update({
#                                 "image_id": image_data["image_id"],
#                                 "image_type": image_data.get("image_type", "unknown")
#                             })
                            
#                             image_results.append(question_result)
                            
#                         except Exception as e:
#                             print(f"Failed question {question['id']}: {e}")
#                             current_image_questions_failed += 1
#                             total_questions_failed += 1
#                             continue
                    
#                     # Add results from this image
#                     results.extend(image_results)
                    
#                     # Calculate success metrics
#                     questions_succeeded = current_image_questions_total - current_image_questions_failed
                    
#                     if current_image_questions_failed == 0:
#                         successful_images.append({
#                             'image_id': image_data['image_id'],
#                             'image_type': image_data.get('image_type', 'unknown'),
#                             'questions_total': current_image_questions_total,
#                             'questions_succeeded': questions_succeeded,
#                             'processing_time': time.time() - image_start_time
#                         })
#                     else:
#                         image_success_rate = (questions_succeeded / current_image_questions_total * 100) if current_image_questions_total > 0 else 0
#                         failed_images.append({
#                             'image_id': image_data['image_id'],
#                             'image_type': image_data.get('image_type', 'unknown'),
#                             'error_type': 'partial_failure',
#                             'questions_total': current_image_questions_total,
#                             'questions_failed': current_image_questions_failed,
#                             'questions_succeeded': questions_succeeded,
#                             'success_rate': f"{image_success_rate:.1f}%"
#                         })
                    
#                     # Ultra-aggressive cleanup after each image
#                     del image
#                     self.ultra_aggressive_memory_cleanup()
                    
#                     # Save intermediate results
#                     if (idx + 1) % 2 == 0 or idx == total_images - 1:
#                         checkpoint_path = f"{save_path}_checkpoint.json"
#                         with open(checkpoint_path, 'w') as f:
#                             json.dump(results, f, indent=4)
                            
#                 except Exception as e:
#                     print(f"Complete failure for image {image_data['image_id']}: {e}")
#                     failed_images.append({
#                         'image_id': image_data['image_id'],
#                         'image_type': image_data.get('image_type', 'unknown'),
#                         'error_type': 'complete_failure',
#                         'error_message': str(e)
#                     })
                    
#                     # Cleanup even on failure
#                     self.ultra_aggressive_memory_cleanup()
#                     continue
            
#             # Save final results
#             if results:
#                 with open(save_path, 'w') as f:
#                     json.dump(results, f, indent=4)
            
#         except Exception as e:
#             print(f"Critical error during evaluation: {e}")
#             if results:
#                 error_save_path = f"{save_path}_error_state.json"
#                 with open(error_save_path, 'w') as f:
#                     json.dump(results, f, indent=4)
        
#         # Print comprehensive summary
#         self._print_evaluation_summary(
#             model_name, total_images, successful_images, failed_images, 
#             total_questions_processed, total_questions_failed, len(results)
#         )
        
#         return results
    
#     def _print_evaluation_summary(self, model_name, total_images, successful_images, 
#                                 failed_images, total_questions_processed, total_questions_failed, total_results):
#         """Print a comprehensive evaluation summary."""
#         print(f"\n{'='*60}")
#         print(f"EVALUATION SUMMARY FOR {model_name.upper()}")
#         print(f"{'='*60}")
        
#         # Image-level statistics
#         num_successful = len(successful_images)
#         num_failed = len(failed_images)
        
#         print(f"📊 IMAGE PROCESSING SUMMARY:")
#         print(f"   Total images attempted: {total_images}")
#         print(f"   Successfully processed: {num_successful} ({num_successful/total_images*100:.1f}%)")
#         print(f"   Failed images: {num_failed} ({num_failed/total_images*100:.1f}%)")
        
#         # Question-level statistics
#         questions_succeeded = total_questions_processed - total_questions_failed
#         print(f"\n📝 QUESTION PROCESSING SUMMARY:")
#         print(f"   Total questions attempted: {total_questions_processed}")
#         print(f"   Successfully processed: {questions_succeeded} ({questions_succeeded/total_questions_processed*100:.1f}%)")
#         print(f"   Failed questions: {total_questions_failed} ({total_questions_failed/total_questions_processed*100:.1f}%)")
#         print(f"   Results saved: {total_results}")
        
#         # Memory usage summary
#         memory_info = self.get_gpu_memory_info()
#         if memory_info:
#             print(f"\n🧠 FINAL MEMORY USAGE:")
#             print(f"   Current allocation: {memory_info['allocated']:.2f} GB ({memory_info['usage_percent']:.1f}%)")
#             print(f"   Peak allocation: {memory_info['max_allocated']:.2f} GB")
#             print(f"   Total GPU memory: {memory_info['total']:.2f} GB")
        
#         # Successful images details
#         if successful_images:
#             print(f"\n✅ SUCCESSFUL IMAGES ({len(successful_images)}):")
#             for img in successful_images:
#                 print(f"   • {img['image_id']} (Type: {img['image_type']}, "
#                       f"Questions: {img['questions_succeeded']}/{img['questions_total']}, "
#                       f"Time: {img['processing_time']:.1f}s)")
        
#         # Failed images details
#         if failed_images:
#             print(f"\n❌ FAILED/PROBLEMATIC IMAGES ({len(failed_images)}):")
#             for img in failed_images:
#                 if img['error_type'] == 'complete_failure':
#                     print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
#                           f"COMPLETE FAILURE: {img.get('error_message', 'Unknown error')}")
#                 elif img['error_type'] == 'partial_failure':
#                     print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
#                           f"PARTIAL: {img['questions_failed']}/{img['questions_total']} failed "
#                           f"({img['success_rate']} success)")
#                 elif img['error_type'] == 'file_not_found':
#                     print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
#                           f"FILE NOT FOUND: {img['error_message']}")
        
#         print(f"{'='*60}\n")

In [6]:
class BenchmarkTester:
    def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        with open(benchmark_path, 'r') as f:
            self.benchmark = json.load(f)
        self.data_dir = data_dir

    def clean_answer(self, answer):
        """Extract number and reasoning from the model's answer."""
        # Try to extract number and reasoning using regex
        import re
        pattern = r'(\d+)\s*\[(.*?)\]'
        match = re.search(pattern, answer)
        
        if match:
            number = match.group(1)
            objects = [obj.strip() for obj in match.group(2).split(',')]
            return {
                "count": number,
                "reasoning": objects
            }
        else:
            # Fallback if format isn't matched
            numbers = re.findall(r'\d+', answer)
            return {
                "count": numbers[0] if numbers else "0",
                "reasoning": []
            }

    def model_generation(self, model_name, model, inputs, processor):
        """Generate answer and decode with greedy decoding."""
        outputs = None  # Initialize outputs to None
        
        if model_name == "Qwen2.5-VL":
            # Explicit greedy decoding parameters
            outputs = model.generate(
                **inputs, 
                max_new_tokens=200,
                do_sample=False,          # Disable sampling for greedy decoding
                temperature=None,         # Not used in greedy decoding
                top_p=None,              # Not used in greedy decoding  
                top_k=None,              # Not used in greedy decoding
                num_beams=1,             # Single beam for greedy decoding
                early_stopping=False,    # Let it generate until max_tokens or EOS
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id
            )
            outputs = [
                out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, outputs)
            ]
            answer = processor.batch_decode(
                outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]
        else:
            print(f"Warning: Unknown model name '{model_name}' in model_generation.")
            answer = ""  # Return an empty string

        return answer, outputs
    
    def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
        results = []
        
        # Initialize tracking variables
        successful_images = []
        failed_images = []
        total_questions_processed = 0
        total_questions_failed = 0
        
        print(f"\nEvaluating {model_name}...")
        print(f"Using device: {self.device}")
        
        # Force garbage collection before starting
        gc.collect()
        torch.cuda.empty_cache()

        try:
            images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
            total_images = len(images)
            
            for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
                image_start_time = time.time()
                current_image_questions_failed = 0
                current_image_questions_total = 0
                
                try:
                    image_path = Path(self.data_dir)/image_data['path']
                    if not image_path.exists():
                        failed_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'error_type': 'file_not_found',
                            'error_message': f'Image not found at {image_path}'
                        })
                        continue
                    
                    # Load and preprocess image
                    image = Image.open(image_path).convert("RGB")
                    image_results = []  # Store results for current image
                    
                    for question_idx, question in enumerate(image_data['questions']):
                        current_image_questions_total += 1
                        total_questions_processed += 1
                        
                        try:
                            messages = [
                                {
                                    "role": "user",
                                    "content": [
                                        {"type": "image", "image": image},
                                        {"type": "text", "text": f"{question['question']} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]"}
                                    ]
                                },
                            ]
                            
                            # Clear cache before processing each question
                            torch.cuda.empty_cache()
                            
                            # Process image and text for Qwen2.5-VL
                            text = processor.apply_chat_template(
                                messages, tokenize=False, add_generation_prompt=True
                            )
                            inputs = processor(
                                text=text,
                                images=image,
                                videos=None,
                                padding=True,
                                return_tensors="pt",
                            ).to("cuda")
                            
                            # Generate answer with greedy decoding
                            with torch.no_grad():
                                answer, outputs = self.model_generation(model_name, model, inputs, processor)
        
                            cleaned_answer = self.clean_answer(answer)
                            
                            image_results.append({
                                "image_id": image_data["image_id"],
                                "image_type": image_data.get("image_type", "unknown"),
                                "question_id": question["id"],
                                "question": question["question"],
                                "ground_truth": question["answer"],
                                "model_answer": cleaned_answer["count"],
                                "model_reasoning": cleaned_answer["reasoning"],
                                "raw_answer": answer,  # Keep raw answer for debugging
                                "property_category": question["property_category"]
                            })
                            
                            # Clear memory
                            del outputs, inputs
                            torch.cuda.empty_cache()
                            
                        except Exception as e:
                            current_image_questions_failed += 1
                            total_questions_failed += 1
                            continue
                    
                    # Add results from this image
                    results.extend(image_results)
                    
                    # Calculate success rate for this image
                    questions_succeeded = current_image_questions_total - current_image_questions_failed
                    
                    if current_image_questions_failed == 0:
                        # All questions succeeded
                        successful_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'questions_total': current_image_questions_total,
                            'questions_succeeded': questions_succeeded,
                            'processing_time': time.time() - image_start_time
                        })
                    else:
                        # Some questions failed
                        image_success_rate = (questions_succeeded / current_image_questions_total * 100) if current_image_questions_total > 0 else 0
                        failed_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'error_type': 'partial_failure',
                            'questions_total': current_image_questions_total,
                            'questions_failed': current_image_questions_failed,
                            'questions_succeeded': questions_succeeded,
                            'success_rate': f"{image_success_rate:.1f}%"
                        })
                    
                    # Save intermediate results only every 2 images or if it's the last image
                    if (idx + 1) % 2 == 0 or idx == total_images - 1:
                        checkpoint_path = f"{save_path}_checkpoint.json"
                        with open(checkpoint_path, 'w') as f:
                            json.dump(results, f, indent=4)
                            
                except Exception as e:
                    failed_images.append({
                        'image_id': image_data['image_id'],
                        'image_type': image_data.get('image_type', 'unknown'),
                        'error_type': 'complete_failure',
                        'error_message': str(e)
                    })
                    continue
            
            # Save final results
            if results:
                with open(save_path, 'w') as f:
                    json.dump(results, f, indent=4)
            
        except Exception as e:
            if results:
                error_save_path = f"{save_path}_error_state.json"
                with open(error_save_path, 'w') as f:
                    json.dump(results, f, indent=4)
        
        # Print comprehensive summary
        self._print_evaluation_summary(
            model_name, total_images, successful_images, failed_images, 
            total_questions_processed, total_questions_failed, len(results)
        )
        
        return results
    
    def _print_evaluation_summary(self, model_name, total_images, successful_images, 
                                failed_images, total_questions_processed, total_questions_failed, total_results):
        """Print a comprehensive evaluation summary."""
        print(f"\n{'='*60}")
        print(f"EVALUATION SUMMARY FOR {model_name.upper()}")
        print(f"{'='*60}")
        
        # Image-level statistics
        num_successful = len(successful_images)
        num_failed = len(failed_images)
        
        print(f"📊 IMAGE PROCESSING SUMMARY:")
        print(f"   Total images attempted: {total_images}")
        print(f"   Successfully processed: {num_successful} ({num_successful/total_images*100:.1f}%)")
        print(f"   Failed images: {num_failed} ({num_failed/total_images*100:.1f}%)")
        
        # Question-level statistics
        questions_succeeded = total_questions_processed - total_questions_failed
        print(f"\n📝 QUESTION PROCESSING SUMMARY:")
        print(f"   Total questions attempted: {total_questions_processed}")
        print(f"   Successfully processed: {questions_succeeded} ({questions_succeeded/total_questions_processed*100:.1f}%)")
        print(f"   Failed questions: {total_questions_failed} ({total_questions_failed/total_questions_processed*100:.1f}%)")
        print(f"   Results saved: {total_results}")
        
        # Successful images details
        if successful_images:
            print(f"\n✅ SUCCESSFUL IMAGES ({len(successful_images)}):")
            for img in successful_images:
                print(f"   • {img['image_id']} (Type: {img['image_type']}, "
                      f"Questions: {img['questions_succeeded']}/{img['questions_total']}, "
                      f"Time: {img['processing_time']:.1f}s)")
        
        # Failed images details
        if failed_images:
            print(f"\n❌ FAILED/PROBLEMATIC IMAGES ({len(failed_images)}):")
            for img in failed_images:
                if img['error_type'] == 'complete_failure':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"COMPLETE FAILURE: {img.get('error_message', 'Unknown error')}")
                elif img['error_type'] == 'partial_failure':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"PARTIAL: {img['questions_failed']}/{img['questions_total']} failed "
                          f"({img['success_rate']} success)")
                elif img['error_type'] == 'file_not_found':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"FILE NOT FOUND: {img['error_message']}")
        
        # Group failed images by type
        if failed_images:
            print(f"\n📈 FAILURE ANALYSIS BY IMAGE TYPE:")
            from collections import defaultdict
            failures_by_type = defaultdict(list)
            for img in failed_images:
                failures_by_type[img['image_type']].append(img)
            
            for img_type, failures in failures_by_type.items():
                print(f"   • {img_type}: {len(failures)} failed images")
                for failure in failures:
                    print(f"     - {failure['image_id']} ({failure['error_type']})")
        
        print(f"{'='*60}\n")

## Test SmolVLM Model

Let's evaluate the SmolVLM2-2.2B-Instruct model

In [7]:
# def test_smolVLM2():
#     from transformers import AutoProcessor, AutoModelForImageTextToText

#     print("Loading smolVLM model...")
    
#     model = AutoModelForImageTextToText.from_pretrained(
#         "HuggingFaceTB/SmolVLM2-2.2B-Instruct",
#         torch_dtype=torch.float16,
#         attn_implementation="flash_attention_2",
#         low_cpu_mem_usage=True,
#         trust_remote_code=True
#     ).to("cuda")

#     processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM2-2.2B-Instruct")

#     ## A bit slow without the flash_attention2 requires ampere gpu's. Better performance in some cases

#     # Optional: Enable memory efficient attention
#     if hasattr(model.config, 'use_memory_efficient_attention'):
#         model.config.use_memory_efficient_attention = True

#     tester = BenchmarkTester()
#     smolVLM_results = tester.evaluate_model(
#         "smolVLM2",
#         model, 
#         processor, 
#         "smolVLM2_results_1.json", 
#         batch_size=25
#     )

#     # Clean up
#     del model, processor
#     torch.cuda.empty_cache()
#     gc.collect()

## Test Qwen2.5-VL

Lets evaluate the Qwen2.5-VL-7B-Instruct model

In [8]:
def test_Qwen2_5VL():
    from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
    from accelerate import Accelerator

    accelerator = Accelerator()
    device = accelerator.device
    
    # default: Load the model on the available device(s)
    # model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    #     "Qwen/Qwen2.5-VL-3B-Instruct", 
    #     load_in_8bit=True, # throws error when .to() is added
    #     torch_dtype=torch.bfloat16, 
    #     device_map="auto",
    #     # attn_implementation="flash_attention_2",
    #     low_cpu_mem_usage=True
    # )
    
    # We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        "/var/scratch/ave303/models/qwen2-5-vl-32b",
        torch_dtype=torch.float16,
        # load_in_8bit=True,
        attn_implementation="flash_attention_2",
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    min_pixels = 256*28*28
    max_pixels = 1280*28*28
    
    # default processer
    processor = AutoProcessor.from_pretrained("/var/scratch/ave303/models/qwen2-5-vl-32b", min_pixels=min_pixels, max_pixels=max_pixels)

    ### Qwen2.5-VL-7B-Instruct --> goes out of CUDA memory
    ### Qwen2.5-VL-3B-Instruct --> can handle only 2 images before going out of memory but decent performance

    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True

    tester = BenchmarkTester()
    Qwen2_5VL_results = tester.evaluate_model(
        "Qwen2.5-VL",
        model, 
        processor, 
        "Qwen2-5-vl-32b_results.json",
        start_idx=0,
        batch_size=360
    )

    # Clean up
    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

## Run Evaluation

Now we can run our evaluation. Let's start with the SmolVLM2 model:

In [9]:
# test_smolVLM2()

In [10]:
test_Qwen2_5VL()

[2025-07-30 16:08:32,726] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: warning: libm.so.6, needed by /cm/shared/apps/cuda12.3/toolkit/12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: /cm/shared/apps/cuda12.3/toolkit/12.3/lib64/libcufile.so: undefined reference to `log2f@GLIBC_2.2.5'
/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: /opt/ohpc/pub/compiler/gcc/12.4.0/lib64/libstdc++.so.6: undefined reference to `fesetround@GLIBC_2.2.5'
/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: /cm/shared/apps/cuda12.3/toolkit/12.3/lib64/libcufile.so: undefined reference to `dlvsym'
/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: /cm/shared/apps/cuda12.3/toolkit/12.3/lib64/libcufile.so: undefined reference to `dlopen'
/var/scratch/ave303/anaconda3/envs/op_bench/compiler_compat/ld: /opt/ohpc/pub/compiler/gcc/12.4.0/lib64/libstdc++.so.6: undefined reference to `fegetround@GLIBC_2.

[2025-07-30 16:08:35,488] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Processing images:  99%|█████████▉| 358/360 [6:35:43<01:51, 55.69s/it]

Processing images: 100%|█████████▉| 359/360 [6:37:17<01:07, 67.24s/it]

Processing images: 100%|██████████| 360/360 [6:38:16<00:00, 64.59s/it]

Processing images: 100%|██████████| 360/360 [6:38:16<00:00, 66.38s/it]


EVALUATION SUMMARY FOR QWEN2.5-VL
📊 IMAGE PROCESSING SUMMARY:
   Total images attempted: 360
   Successfully processed: 360 (100.0%)
   Failed images: 0 (0.0%)

📝 QUESTION PROCESSING SUMMARY:
   Total questions attempted: 1080
   Successfully processed: 1080 (100.0%)
   Failed questions: 0 (0.0%)
   Results saved: 1080

✅ SUCCESSFUL IMAGES (360):
   • image01 (Type: REAL, Questions: 3/3, Time: 59.2s)
   • image02 (Type: REAL, Questions: 3/3, Time: 83.7s)
   • image03 (Type: REAL, Questions: 3/3, Time: 72.9s)
   • image04 (Type: REAL, Questions: 3/3, Time: 86.9s)
   • image05 (Type: REAL, Questions: 3/3, Time: 77.4s)
   • image06 (Type: REAL, Questions: 3/3, Time: 92.0s)
   • image07 (Type: REAL, Questions: 3/3, Time: 44.6s)
   • image08 (Type: REAL, Questions: 3/3, Time: 32.0s)
   • image09 (Type: REAL, Questions: 3/3, Time: 78.9s)
   • image10 (Type: REAL, Questions: 3/3, Time: 116.8s)
   • image11 (Type: REAL, Questions: 3/3, Time: 48.9s)
   • image12 (Type: REAL, Questions: 3/3, Ti